## LSSTCam Image Header Check

In [ ]:
import logging
import re
import sys

from astropy.time import Time, TimeDelta
import numpy as np

from lsst.daf.butler import Butler
from lsst_efd_client import EfdClient
from lsst.ts.xml.enums import MTDomeTrajectory as eMTDT
from lsst.ts.xml.enums import MTPtg as eMTPtg

import lsst.sitcom.mareuter.header_checks as hc
import lsst.sitcom.mareuter.header_utils as hu
import lsst.sitcom.mareuter.site_efd as se

run_logging = False

In [ ]:
if run_logging:
    stream_handler = logging.StreamHandler(sys.stdout)

    logger = logging.getLogger()
    logger.addHandler(stream_handler)
    logger.level = logging.NOTSET
    logging.getLogger("matplotlib").setLevel(logging.WARNING)

In [ ]:
efd_name = se.get_efd()
collections = ["LSSTCam/raw/all", "LSSTCam/calib/unbounded"]
instrument = "LSSTCam"
butler = Butler(instrument, collections=collections, instrument=instrument)
client = EfdClient(efd_name)

In [ ]:
day_obs = 20250412
seq_num = 22
raft_name = "R22"
sensor_name = "S11"
ccd_name = f"{raft_name}_{sensor_name}"
ccd_loc = f"{raft_name}{sensor_name}"

In [ ]:
camera = butler.get("camera", instrument=instrument)
det = camera.getNameMap()[ccd_name]
detector = det.getId()

In [ ]:
dataId = {"instrument": instrument, "detector": detector,
          "exposure.day_obs": day_obs, "exposure.seq_num": seq_num}
raw = butler.get('raw.metadata', dataId)

In [ ]:
header = raw.toDict()
#print(list(header.keys()))
# print(header)

In [ ]:
delta = TimeDelta(20.0, format="sec", scale="tai")
obs_beg = Time(header["DATE-BEG"], format="fits", scale="tai")
obs_end = Time(header["DATE-END"], format="fits", scale="tai")
exp_time = TimeDelta(float(header["EXPTIME"]), format="sec", scale="tai")
#print(obs_beg)
#print(header["DATE-BEG"])
obs_id = header["OBSID"]
print(f"DATE-BEG: {header['DATE-BEG']}")
print(f"DATE-END: {header['DATE-END']}")

In [ ]:
start_int_df = await client.select_time_series("lsst.sal.MTCamera.logevent_startIntegration", "*",
                                               obs_beg.utc, delta, is_window=True)
start_readout_df =  await client.select_time_series("lsst.sal.MTCamera.logevent_startReadout", "*",
                                               obs_end.utc, delta, is_window=True)
end_image_tel_df = await client.select_time_series("lsst.sal.MTCamera.logevent_endOfImageTelemetry", "*",
                                                   obs_end.utc, delta, is_window=True)

In [ ]:
print(f"Start Integration Num Rows: {len(start_int_df)}")
print(f"Start Readout Num Rows: {len(start_readout_df)}")
print(f"End of Image Telemetry Num Rows: {len(end_image_tel_df)}")

In [ ]:
start_int = start_int_df.loc[start_int_df["imageName"] == obs_id]
start_ro = start_readout_df.loc[start_readout_df["imageName"] == obs_id]
end_image_tel = end_image_tel_df.loc[end_image_tel_df["imageName"] == obs_id]

In [ ]:
# Scales are UTC because incoming timestamps are already TAI and astropy screws them up if the scale is TAI.
date_obs = Time(end_image_tel["timestampDateObs"].iloc[0], format="unix_tai", scale="tai")
date_end = Time(end_image_tel["timestampDateEnd"].iloc[0], format="unix_tai", scale="tai")
date_start_int = Time(start_int["private_sndStamp"].iloc[0], format="unix_tai", scale="tai").utc
date_start_ro = Time(start_ro["private_sndStamp"].iloc[0], format="unix_tai", scale="tai").utc
date_eoit = Time(end_image_tel["private_sndStamp"].iloc[0], format="unix_tai", scale="tai").utc
print(date_start_int.isot)
print(date_eoit.isot)

In [ ]:
hc.check("DATE-BEG", header, date_obs.isot)
hc.check("DATE-END", header, date_end.isot)

In [ ]:
date_diff = date_end - date_obs
if date_diff.sec >= float(header["EXPTIME"]):
    print("DATE-END - DATE-BEG OK")
else:
    print(f"Problem with DATE-END - DATE-BEG: {date_diff.sec} seconds")
    print(f"Exposure time: {header['EXPTIME']} seconds")

In [ ]:
image_readout_params_df = await client.select_top_n("lsst.sal.MTCamera.logevent_imageReadoutParameters",
                                                    "*", 1,
                                                    time_cut=date_start_int.isot)

In [ ]:
hc.check("OBSID", header, hu.get_value(start_int, "imageName"))
hc.check("CAMCODE", header, hu.get_value(start_int, "imageSource"))
hc.check("CONTRLLR", header, hu.get_value(start_int, "imageController"))
hc.check("DAYOBS", header, hu.get_value(start_int, "imageDate"))
hc.check("SEQNUM", header, hu.get_value(start_int, "imageNumber"))
take_image_kv = hu.dict_from_additional(start_int)
hc.check_take_image("GROUPID", header, take_image_kv, "groupId")
hc.check_take_image("IMGTYPE", header, take_image_kv, "imageType")
hc.check_take_image("TESTTYPE", header, take_image_kv, "testType")
hc.check_take_image("PROGRAM", header, take_image_kv, "program")
hc.check_take_image("REASON", header, take_image_kv, "reason")
hc.check("OBSANNOT", header, hu.get_value(image_readout_params_df, "daqAnnotation"))
hc.check("CURINDEX", header, hu.get_value(start_int, "imageIndex"))
hc.check("MAXINDEX", header, hu.get_value(start_int, "imagesInSequence"))
hc.check_not_empty("DETSIZE", header)
# hc.check("OVERH", header, hu.get_value(image_readout_params_df, "overCols"))
# hc.check("OVERV", header, hu.get_value(image_readout_params_df, "overRows"))
# hc.check("PREH", header, hu.get_value(image_readout_params_df, "preCols"))
hc.check_float("EXPTIME", header, hu.get_value(start_int, "exposureTime"))
hc.check_float("DARKTIME", header, hu.get_value(end_image_tel, "darkTime"), precision=1e-5)
hc.check_float("SHUTTIME", header, hu.get_value(end_image_tel, "measuredShutterOpenTime"))

In [ ]:
current_target_df = await client.select_top_n("lsst.sal.MTPtg.logevent_currentTarget", "*", 1,
                                              time_cut=date_start_int.isot)
current_target_status_start_df = await client.select_top_n("lsst.sal.MTPtg.currentTargetStatus", ["airmass", "ha"], 1,
                                                           time_cut=date_start_int.isot)
current_target_status_end_df = await client.select_top_n("lsst.sal.MTPtg.currentTargetStatus", ["airmass", "ha"], 1,
                                                         time_cut=date_start_ro.isot)
mount_positions_start_df = await client.select_top_n("lsst.sal.MTPtg.mountPosition", ["ra", "declination", "skyAngle"], 1,
                                                     time_cut=date_start_int.isot)
mount_positions_end_df = await client.select_top_n("lsst.sal.MTPtg.mountPosition", ["ra", "declination"], 1,
                                                   time_cut=date_eoit.isot)

In [ ]:
mount_az_start_df = await client.select_top_n("lsst.sal.MTMount.azimuth", "actualPosition", 1,
                                              time_cut=date_start_int.isot)
mount_az_end_df = await client.select_top_n("lsst.sal.MTMount.azimuth", "actualPosition", 1,
                                            time_cut=date_start_ro.isot)
mount_el_start_df = await client.select_top_n("lsst.sal.MTMount.elevation", "actualPosition", 1,
                                              time_cut=date_start_int.isot)
mount_el_end_df = await client.select_top_n("lsst.sal.MTMount.elevation", "actualPosition", 1,
                                            time_cut=date_start_ro.isot)

In [ ]:
hexapod_app_df = await client.select_top_n("lsst.sal.MTHexapod.application", "demand2", 1,
                                           time_cut=date_start_int.isot, index=1)

In [ ]:
vignette_range_df = await client.select_time_series("lsst.sal.MTDomeTrajectory.logevent_telescopeVignetted", "vignetted",
                                              start=date_start_int, end=date_eoit)
vignette_last_df = await client.select_top_n("lsst.sal.MTDomeTrajectory.logevent_telescopeVignetted", "vignetted", 1,
                                             time_cut=date_start_int.isot)

In [ ]:
hc.check("OBJECT", header, hu.get_value(current_target_df, "targetName"))
hc.check_float("ROTPA", header, hu.get_value(mount_positions_start_df, "skyAngle"))
hc.check("ROTCOORD", header, "sky")
# if header["IMGTYPE"] in ["OBJECT", "ENGTEST", "ACQ"]:
hc.check_float("RA", header, np.degrees(hu.get_value(current_target_df, "ra")))
hc.check_float("DEC", header, np.degrees(hu.get_value(current_target_df, "declination")))
hc.check_float("RASTART", header, hu.get_value(mount_positions_start_df, "ra"))
hc.check_float("RAEND", header, hu.get_value(mount_positions_end_df, "ra"))
hc.check_near("RA", "RASTART", header, 0.5)
hc.check_near("RA", "RAEND", header, 0.5)
hc.check_float("DECSTART", header, hu.get_value(mount_positions_start_df, "declination"))
hc.check_float("DECEND", header, hu.get_value(mount_positions_end_df, "declination"))
hc.check_near("DEC", "DECSTART", header, 0.1)
hc.check_near("DEC", "DECEND", header, 0.1)
hc.check_float("HASTART", header, hu.get_value(current_target_status_start_df, "ha"))
hc.check_float("HAEND", header, hu.get_value(current_target_status_end_df, "ha"))
hc.check_float("AMSTART", header, hu.get_value(current_target_status_start_df, "airmass"))
hc.check_float("AMEND", header, hu.get_value(current_target_status_end_df, "airmass"))
# else:
    # print(f"No check on RA/DEC for {header['IMGTYPE']}")
hc.check_float("AZSTART", header, hu.get_value(mount_az_start_df, "actualPosition"))
hc.check_float("AZEND", header, hu.get_value(mount_az_end_df, "actualPosition"))
hc.check_float("ELSTART", header, hu.get_value(mount_el_start_df, "actualPosition"))
hc.check_float("ELEND", header, hu.get_value(mount_el_end_df, "actualPosition"))
hc.check_float("FOCUSZ", header, hu.get_value(hexapod_app_df, "demand2"))
hc.check_enum("TRACKSYS", header, eMTPtg.TargetTypes, hu.get_value(current_target_df, "targetType"))
hc.check_enum("RADESYS", header, eMTPtg.CoordFrame, hu.get_value(current_target_df, "frame"))
hc.check_enum("VIGNETTE", header, eMTDT.TelescopeVignetted, hu.get_monitor_value(vignette_last_df, vignette_range_df, "vignetted", "max"))
hc.check_enum("VIGN_MIN", header, eMTDT.TelescopeVignetted, hu.get_monitor_value(vignette_last_df, vignette_range_df, "vignetted", "min"))

In [ ]:
sim_mtmount_df = await client.select_top_n("lsst.sal.MTMount.logevent_simulationMode", "mode", 1,
                                           time_cut=date_start_int.isot)
sim_mtm1m3_df = await client.select_top_n("lsst.sal.MTM1M3.logevent_simulationMode", "mode", 1,
                                          time_cut=date_start_int.isot)
sim_mtm2_df = await client.select_top_n("lsst.sal.MTM2.logevent_simulationMode", "mode", 1,
                                        time_cut=date_start_int.isot)
sim_mtcamhex_df = await client.select_top_n("lsst.sal.MTHexapod.logevent_simulationMode", "mode", 1,
                                            time_cut=date_start_int.isot, index=1)
sim_mtm2hex_df = await client.select_top_n("lsst.sal.MTHexapod.logevent_simulationMode", "mode", 1,
                                           time_cut=date_start_int.isot, index=2)
sim_mtrotator_df = await client.select_top_n("lsst.sal.MTRotator.logevent_simulationMode", "mode", 1,
                                             time_cut=date_start_int.isot)
sim_mtdome_df = await client.select_top_n("lsst.sal.MTDome.logevent_simulationMode", "mode", 1,
                                          time_cut=date_start_int.isot)
sim_mtdometraj_df = await client.select_top_n("lsst.sal.MTDomeTrajectory.logevent_simulationMode", "mode", 1,
                                              time_cut=date_start_int.isot)

In [ ]:
hc.check("HIERARCH SIMULATE MTMOUNT", header, hu.get_value(sim_mtmount_df, "mode"))
# try:
#     value = sim_mtm1m3_df["mode"].iloc[0]
# except KeyError:
#     value = "None"
hc.check("HIERARCH SIMULATE MTM1M3", header, hu.get_value(sim_mtm1m3_df, "mode"))
hc.check("HIERARCH SIMULATE MTM2", header, hu.get_value(sim_mtm2_df, "mode"))
hc.check("HIERARCH SIMULATE CAMHEXAPOD", header, hu.get_value(sim_mtcamhex_df, "mode"))
hc.check("HIERARCH SIMULATE M2HEXAPOD", header, hu.get_value(sim_mtm2hex_df, "mode"))
hc.check("HIERARCH SIMULATE MTROTATOR", header, hu.get_value(sim_mtrotator_df, "mode"))
hc.check("HIERARCH SIMULATE MTDOME", header, hu.get_value(sim_mtdome_df, "mode"))
hc.check("HIERARCH SIMULATE MTDOMETRAJECTORY", header, hu.get_value(sim_mtdometraj_df, "mode"))

In [ ]:
filter_pos_df = await client.select_top_n("lsst.sal.MTCamera.logevent_endSetFilter", "*", 1,
                                          time_cut=date_start_int.isot)

In [ ]:
hc.check("FILTBAND", header, hu.get_value(filter_pos_df, "filterType"))
hc.check("FILTER", header, hu.get_value(filter_pos_df, "filterName"))
hc.check("FILTSLOT", header, hu.get_value(filter_pos_df, "filterSlot"))
hc.check("FILTPOS", header, hu.get_value(filter_pos_df, "filterPosition"))

In [ ]:
focal_plane_info_df = await client.select_top_n("lsst.sal.MTCamera.logevent_focalPlaneSummaryInfo", "*", 1,
                                                time_cut=date_start_int.isot)
fp_ccd_df = await client.select_top_n("lsst.sal.MTCamera.focal_plane_Ccd", "*", 1,
                                      time_cut=date_start_int.isot)

In [ ]:
cdetector = focal_plane_info_df["ccdLocation"].iloc[0].split(":").index(ccd_loc)
hc.check("CCD_MANU", header, hu.get_value(focal_plane_info_df, "ccdManufacturer"), index=cdetector)
hc.check("CCD_TYPE", header, hu.get_value(focal_plane_info_df, f"ccdType{cdetector}"))
hc.check("CCD_SERN", header, hu.get_value(focal_plane_info_df, "ccdManSerNum"), index=cdetector)
hc.check("LSST_NUM", header, hu.get_value(focal_plane_info_df, "ccdLSSTName"), index=cdetector)
hc.check("CCDSLOT", header, hu.get_value(focal_plane_info_df, "ccdSlot"), index=cdetector)
hc.check("RAFTBAY", header, hu.get_value(focal_plane_info_df, "raftBay"), index=cdetector)
hc.check("SEQCKSUM", header, hu.get_value(focal_plane_info_df, "sequencerChecksum"), index=cdetector)
hc.check("SEQNAME", header, hu.get_value(focal_plane_info_df, "sequencerKey"), index=cdetector)
hc.check("REBNAME", header, hu.get_value(focal_plane_info_df, "rebLSSTName"), index=cdetector)
hc.check("CONTNUM", header, hu.get_value(focal_plane_info_df, "rebSerialNumber"), index=cdetector) 
hc.check_float("CCDTEMP", header, hu.get_value(fp_ccd_df, f"temp{cdetector}"))
hc.check_float("TEMP_SET", header, hu.get_value(focal_plane_info_df, f"ccdTempSetPoint{cdetector}"))
hc.check("IMAGETAG", header, hu.get_value(end_image_tel, "imageTag"))

In [ ]:
temp_df = await client.select_top_n("lsst.sal.ESS.temperature", "temperatureItem0", 1,
                                       time_cut=date_start_int.isot, index=301)
pressure_df = await client.select_top_n("lsst.sal.ESS.pressure", "pressureItem0", 1,
                                        time_cut=date_start_int.isot, index=301)
humidity_df = await client.select_top_n("lsst.sal.ESS.relativeHumidity", "relativeHumidityItem", 1,
                                        time_cut=date_start_int.isot, index=301)
wind_df = await client.select_top_n("lsst.sal.ESS.airFlow", ["speed", "direction"], 1,
                                          time_cut=date_start_int.isot, index=301)
seeing_df = await client.select_top_n("lsst.sal.DIMM.logevent_dimmMeasurement", "*", 1,
                                      time_cut=date_start_int.isot, index=1)
# dome_pos_df = await client.select_top_n("lsst.sal.MTDome.position", "*", 1,
#                                         time_cut=date_start_int.isot)

In [ ]:
hc.check_float("AIRTEMP", header, hu.get_value(temp_df, "temperatureItem0"))
hc.check_float("PRESSURE", header, hu.get_value(pressure_df, "pressureItem0"))
hc.check_float("HUMIDITY", header, hu.get_value(humidity_df, "relativeHumidityItem"))
hc.check_float("WINDSPD", header, hu.get_value(wind_df, "speed"))
hc.check_float("WINDDIR", header, hu.get_value(wind_df, "direction"))
hc.check_float("SEEING", header, hu.get_value(seeing_df, "fwhm"))

In [ ]:
hc.check("FACILITY", header, "Vera C. Rubin Observatory")
hc.check_not_empty("DATE", header)
hc.check_not_empty("MJD-OBS", header)
hc.check_not_empty("MJD-BEG", header)
hc.check_not_empty("MJD-END", header)
hc.check("BUNIT", header, "adu")
hc.check("TELESCOP", header, "Simonyi Survey Telescope")
hc.check("INSTRUME", header, "lsstCam")
hc.check("OBSERVER", header, "LSST")
hc.check_float("OBS-LONG", header, -70.749417)
hc.check_float("OBS-LAT", header, -30.244639)
hc.check_float("OBS-ELEV", header, 2663.0)
hc.check_float("OBSGEO-X", header, 1818938.94)
hc.check_float("OBSGEO-Y", header, -5208470.95)
hc.check_float("OBSGEO-Z", header, -3195172.08)
hc.check_not_empty("FILENAME", header)
hc.check_not_empty("HEADVER", header)
hc.check_not_empty("TSTAND", header)
hc.check_not_empty("CHECKSUM", header)
hc.check_not_empty("DATASUM", header)
hc.check("TIMESYS", header, "TAI")

In [ ]:
# Segment headers
hc.check("XTENSION", header, "IMAGE")
hc.check("BITPIX", header, 8)
hc.check("INHERIT", header, True)
hc.check("PCOUNT", header, 0)
hc.check("GCOUNT", header, 1)
hc.check_not_empty("EXTNAME", header)
hc.check_not_empty("DATASEC", header)
hc.check_not_empty("DETSEC", header)
hc.check_not_empty("DTV1", header)
hc.check_not_empty("DTV2", header)
hc.check_not_empty("DTM1_1", header)
hc.check_not_empty("DTM1_2", header)
hc.check_not_empty("DTM2_1", header)
hc.check_not_empty("DTM2_2", header)